In [1]:
# imports
import imageio
import matplotlib.pyplot as plt
import os
import numpy as np
import h5py

from pysteps.visualization.precipfields import get_colormap

Pysteps configuration file found at: /vol/knmimo-nobackup/users/pkools/thesis-forecasting/Rainformer/.rainformer_venv/lib/python3.10/site-packages/pysteps/pystepsrc



In [4]:
dir_rtcor = '/vol/knmimo-nobackup/restore/knmimo/thesis_pelle/data/dataset_rtcor_complete/'
prefix_rtcor_recent = "RAD_NL25_RAC_RT_"

output_dir = "../results/figures"
os.makedirs(output_dir, exist_ok=True)

cmap, norm, _, _ = get_colormap('intensity', 'mm/h', 'pysteps')

In [ ]:
# gt_img = f['image1']['image_data'][:]

# ## Set pixels out of image to 0
# out_of_image = f['image1']['calibration'].attrs['calibration_out_of_image']
# gt_img[gt_img == out_of_image] = 0
# # Sometimes 255 or other number (244) is used for the calibration
# # for out of image values, so also check the first pixel
# gt_img[gt_img == gt_img[0][0]] = 0
# # set masked values to 0
# gt_img[gt_img == 65535] = 0
# # Expand dimensions from (w,h) to (w,h,c=1)
# # gt_img = np.expand_dims(gt_img, axis=-1)

# gt_img = (gt_img/100)*12

In [7]:
timestamps_output = ['202301010120', '202301010125', '202301010130', '202301010135', '202301010140', '202301010145', '202301010150', '202301010155', '202301010200']
gt_images = []
start_ts = timestamps_output[0]
for ts in timestamps_output:
    year_dir = os.path.join(dir_rtcor, ts[:4])
    filepath = os.path.join(year_dir, f'{prefix_rtcor_recent}{ts}.h5')
    with h5py.File(filepath, 'r') as f:
        gt_img = f['image1']['image_data'][:]

        out_of_image = f['image1']['calibration'].attrs['calibration_out_of_image']
        mask = (gt_img != out_of_image) & (gt_img != gt_img[0][0]) & (gt_img != 65535)

        gt_img = (gt_img/100)*12

        fig, ax = plt.subplots(figsize=(700 / 100, 765 / 100), dpi=100)  # Ensure 1:1 pixel representation
        img = np.nan_to_num(gt_img)
        img[~mask] = np.nan
        ax.imshow(img, cmap=cmap, norm=norm, interpolation='nearest')  # 'nearest' prevents smoothing
        ax.set_xlim(0, 700)
        ax.set_ylim(765, 0)
        ax.axis('off')
        fig.canvas.draw()

        # Convert plot to image array
        image = np.frombuffer(fig.canvas.buffer_rgba(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (4,))
        gt_images.append(image)
        plt.close(fig)

gif_path = os.path.join(output_dir, f"gt_sample_{start_ts}.gif")
imageio.mimsave(gif_path, gt_images, fps=1)

/tmp/ipykernel_1430562/1541438240.py:23: MatplotlibDeprecationWarning: The tostring_rgb function was deprecated in Matplotlib 3.8 and will be removed in 3.10. Use buffer_rgba instead.
  image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
